In [27]:
import pandas as pd
import glob
import os
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import filepath 
import infrastructure
import pygsheets 
import requests
import smartsheet
from requests.structures import CaseInsensitiveDict
from os import listdir
from os.path import isfile, join
import datetime as dt
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
current_date = now.strftime("%y%m%d")
print("Last update time:", current_time)
import json
import time

Last update time: 02/22/24 10:32:40


In [28]:
filepath.smartsheet_folder

'D:\\Users\\lilig\\Desktop\\SMS-NewData\\smartsheet'

In [29]:
offer_sms = infrastructure.get_smartsheet('offers_sms')
test_sms = infrastructure.get_smartsheet('test_sms')

In [30]:
## Enter the data foler you place output the data file 
output_filepath = filepath.output_folder
## Enter the data foler you place store the input data 
input_filepath = filepath.input_folder
Segment_filepath = filepath.output_folder+'Segment/'
smsreport_filepath = filepath.output_folder+'SMS_Report/'
smsapikey_filepath = filepath.output_folder+'SMS_Apikey/'

In [31]:

offer_sms1 = offer_sms[offer_sms['Hitpath Offer ID'].isna()== False]
#offer_sms1['LC Offer'] =  offer_sms1['Hitpath Offer ID'].astype('str').str.split(".",expand = True)[0] + '_'+offer_sms1['Abbreviation'].str.split('_',expand = True)[1] 
#offer_sms1 = offer_sms1[offer_sms1['LC Offer'].isna()== False]

In [32]:
#offer_sms1['LC Offer'].to_csv('LC_offer.csv')

# GET User data 

In [33]:
"""
# If you want to get the jump page click data from a specific date range 
start_date = '2023-05-01'
end_date = '2023-08-22'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)

url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
with open(output_filepath + 'User_information'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 

 """

'\n# If you want to get the jump page click data from a specific date range \nstart_date = \'2023-05-01\'\nend_date = \'2023-08-22\'\n# start_date  \ndate_range = pd.date_range(start=start_date, end=end_date)\n\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"\nresp = requests.get(url)\nwith open(output_filepath + \'User_information\'+current_date+\'.csv\', \'w+\') as f:\n    f.write(resp.text)\n# The code I downlodaed in the trinity, I want to combine it all in one files \n## Get the jump page clicks \n\n '

## Get Jump Page Clicks Number

In [34]:
""" 
# There's maximum 45 period windows to pull the  jump page clicks from API 
# Manual download the jump page click file from API
first_date_of_month = pd.to_datetime('2023-10-01')
last_date_of_month = pd.to_datetime('2023-10-31')
first_date_of_month_url = first_date_of_month.strftime('%m/%d/%Y')
yesterday = last_date_of_month.strftime('%m/%d/%Y')
current_date = first_date_of_month.strftime("%y%m")
url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
time.sleep(10)
with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
"""

' \n# There\'s maximum 45 period windows to pull the  jump page clicks from API \n# Manual download the jump page click file from API\nfirst_date_of_month = pd.to_datetime(\'2023-10-01\')\nlast_date_of_month = pd.to_datetime(\'2023-10-31\')\nfirst_date_of_month_url = first_date_of_month.strftime(\'%m/%d/%Y\')\nyesterday = last_date_of_month.strftime(\'%m/%d/%Y\')\ncurrent_date = first_date_of_month.strftime("%y%m")\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"\nresp = requests.get(url)\ntime.sleep(10)\nwith open(output_filepath + \'Jump Page Click from API/jumppage_click\'+current_date+\'.csv\', \'w+\') as f:\n    f.write(resp.text)\n'

In [35]:

#import new mamba 
# get active pubid 
mamba = infrastructure.get_mamba()
mamba['PUBID'] = mamba['Dataset'].str.split('_',expand = True)[2]
active_pubid =','.join(mamba['PUBID'].unique().tolist())
active_pubid += ',460001'
# yesterday
yesterday = date.today() - timedelta(days=1)
first_date_of_month = yesterday.replace(day=1)
yesterday = yesterday.strftime('%m/%d/%Y')
first_date_of_month_url = first_date_of_month.strftime('%m/%d/%Y')
current_date =  first_date_of_month.strftime("%y%m")

## get last month's first date and last date 
lastMonth_last_date = first_date_of_month - timedelta(days=1)
lastMonth_first_date = lastMonth_last_date.replace(day=1)
first_date_of_last_month_url = lastMonth_first_date.strftime('%m/%d/%Y')
last_date_last_month_url = lastMonth_last_date.strftime('%m/%d/%Y')
lastmonth_date =  lastMonth_first_date.strftime("%y%m")

url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
time.sleep(10)
with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
if date.today().day < 10:
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={active_pubid}"
    resp = requests.get(url)
    time.sleep(10)
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+lastmonth_date+'.csv', 'w+') as f:
        f.write(resp.text)

"""
# If you want to get the jump page click data from a specific date range 
start_date = '2023-09-01'
end_date = '2023-09-01'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)
for date in date_range:
    current_date_url = date.strftime('%m/%d/%Y')
    current_date = date.strftime("%y%m%d")
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}"
    resp = requests.get(url)
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
        f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 

os.chdir(output_filepath+'Jump Page Click from trinity')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202211-20230516.csv", index=False, encoding='utf-8-sig')

"""

 

D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


'\n# If you want to get the jump page click data from a specific date range \nstart_date = \'2023-09-01\'\nend_date = \'2023-09-01\'\n# start_date  \ndate_range = pd.date_range(start=start_date, end=end_date)\nfor date in date_range:\n    current_date_url = date.strftime(\'%m/%d/%Y\')\n    current_date = date.strftime("%y%m%d")\n    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}"\n    resp = requests.get(url)\n    with open(output_filepath + \'Jump Page Click from API/jumppage_click\'+current_date+\'.csv\', \'w+\') as f:\n        f.write(resp.text)\n# The code I downlodaed in the trinity, I want to combine it all in one files \n## Get the jump page clicks \n\nos.chdir(output_filepath+\'Jump Page Click from trinity\')\n\nall_files = [i for i in glob.glob(\'*.{}\'.format(\'csv\'))]\n#combine all files in the list\ncombined_jump_page_csv = pd.concat([pd.read_csv(f) for 

In [36]:
os.chdir(output_filepath+'jumppageclicks202309-202310')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click2023091-202310.csv", index=False, encoding='utf-8-sig')

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2640\3855447747.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2640\3855447747.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2640\3855447747.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2640\3855447747.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])


In [37]:
print(url)

http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=02/01/2024&end=02/21/2024&agents=461263,461681,460918,460654,461313,461653,461500,461795,461794,461810,461452,461242,461247,461288,461227,461680,461840,461123,461838,461842,461843,461835,460001


In [38]:
os.chdir(output_filepath+'Jump Page Click from API')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'})
api_jump['subid_1'] = api_jump['subid_1'].fillna('').astype(str)
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})')
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")
jump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])
jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks')
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist()
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
#added errors = coerce to passby format error
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y', errors = 'coerce') 
#jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date']) 
#jump_page_clicks['date'] = jump_page_clicks['date'].dt.strftime('%d%b%y')
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig')
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows
# if there's duplicates rows, please check!

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2640\4159054697.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]


,date,subid_1,campaign_id,affiliate_id,hitid
1,NaN,,NaN,NaN,NaN
2,NaN,,NaN,NaN,NaN
3,NaN,,NaN,NaN,NaN
4,NaN,,NaN,NaN,NaN
5,NaN,,NaN,NaN,NaN
0,NaN,,NaN,NaN,NaN
1,NaN,,NaN,NaN,NaN
2,NaN,,NaN,NaN,NaN
3,NaN,,NaN,NaN,NaN
4,NaN,,NaN,NaN,NaN


In [39]:
jump_page_clicks['Date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
# added errors = coerce to bypass format errors
jump_page_clicks['Date'] = pd.to_datetime(jump_page_clicks['Date'], format='%d%b%y', errors = 'coerce') 
jump_page_clicks

,affiliate_id,campaign_id,subid_1,Jump Page Clicks,date,Date
6,460654,4851.0,MP_HZB_460654_CT4713_I.RC_C15_6264_44345_18Aug...,1,2021-08-18,2021-08-18
7,460654,4851.0,SS_SC_HZB_44345_460654_418966_6264_146928_8Nov...,3,2022-11-08,2022-11-08
8,460654,4851.0,SS_SC_HZB_44345_460654_418966_6264_295469_8Nov...,7,2022-11-08,2022-11-08
9,460654,4851.0,SS_SC_HZB_44345_460654_420034_4851_296307_9Nov...,1,2022-11-09,2022-11-09
10,460654,4851.0,SS_SC_HZB_44345_460654_420034_4851_296307_9Nov...,1,2022-11-09,2022-11-09
...,...,...,...,...,...,...
2240056,461842,10086.0,SS_SC_UAA_79743_461842_AR0_10086_563445_1Dec23...,1,2023-12-01,2023-12-01
2240057,461842,10086.0,SS_SC_UAA_79743_461842_AR0_10086_563445_1Dec23...,1,2023-12-01,2023-12-01
2240058,461842,10086.0,SS_SC_UAA_79743_461842_AR0_10086_563445_1Dec23...,1,2023-12-01,2023-12-01
2240059,461842,10086.0,SS_SC_UAA_79743_461842_AR0_10086_563445_1Dec23...,1,2023-12-01,2023-12-01


In [40]:

useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()
useful_click+useless_clicks -  len(jump_page) 
# the missing 502 clicks is from problematic subid_1 
# useless clicks is also from porblematic clicks 

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2640\322886987.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2640\322886987.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()


-5093

## Update Segment

In [41]:

from os import listdir
from os.path import isfile, join
import datetime as dt

mamba = infrastructure.get_mamba()
current_active_segment = mamba.loc[mamba['Date'] == pd.to_datetime(date.today()),'Segment '].unique().tolist()
secondary_segment =  mamba.loc[(mamba['Date'] == pd.to_datetime(date.today())) & (mamba['Segment '].str.contains('TMO|VZN',na = False)),'Segment '].unique().tolist()

                                                                 
segment_files = [f for f in listdir(Segment_filepath) if isfile(join(Segment_filepath, f)) & ('SMS_SC_SS_Segments' in f)]
output_segment = pd.DataFrame()
segment_files.sort()
for i in segment_files: 
    segment_report = pd.read_csv(Segment_filepath+i)
    segment_report['Date'] = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)
    output_segment = pd.concat([updated_segment_report,output_segment], axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
    print(output_segment)
""" 
most_recent_segment_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in segment_files])
segment_year = most_recent_segment_date.year
segment_month = most_recent_segment_date.month
segement_day = most_recent_segment_date.day

most_recent_segment_file = "SMS_SC_SS_Segments{0}{1:0>2}{2:0>2}.csv".format(segment_year,segment_month,segement_day)

segment_report = pd.read_csv( Segment_filepath+ most_recent_segment_file)

segment_report['Date'] = pd.to_datetime(dt.datetime.today().strftime("%D"))
updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)

legacy_segment = pd.read_csv(output_filepath+'SMS_SC_SS_Segments.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_segment.to_csv(input_filepath+'SMS_SC_SS_Segments_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_segment['Date'] = pd.to_datetime(legacy_segment['Date'])
min_date = updated_segment_report['Date'].min()
legacy_segment = legacy_segment[legacy_segment['Date']<min_date]
full_segment = [legacy_segment, updated_segment_report]
output_segment = pd.concat(full_segment, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_segment)
"""
## rename on 5/24
output_segment['name'] = output_segment['name'].str.replace('FLC_I.CC_OI.4P_30DC','FLC_I.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.CR_OI.4P_30DC','FLC_SPK.CR_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_SPK.SWP_OI.4P_30DC','MBC_SPK.SWP_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_AI.CC_OI.4P_30DC','FLC_AI.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_EDM.247L_OI.4P_30DC','FLC_EDM.247L_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_NPD.RTO_OI.4P_30DC','MBC_NPD.RTO_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_PN.FC_OI.4P_30DC','MBC_PN.FC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_WWM.YFA.2_OI.4P_30DC','FLC_WWM.YFA.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_ZM.PL.2_OI.4P_30DC','FLC_ZM.PL.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.SWP_OI.4P.VZN_30DC','FLC_SPK.SWP_30DC_VZN')
output_segment['name'] = output_segment['name'].str.replace('MBC_EDM.247L_OI.4P.TMO_30DC','MBC_EDM.247L_30DC_TMO')
output_segment['name'] = output_segment['name'].str.replace('MBC_WWM.YFA.2_OI.4P_30DC','MBC_WWM.YFA.2_30DC_TMO')
output_segment['Secondary Segment'] = False
output_segment['Active Segment'] = False
output_segment.loc[output_segment['name'].isin(secondary_segment),'Secondary Segment'] = True
output_segment.loc[output_segment['name'].isin(current_active_segment),'Active Segment'] = True

output_segment.to_csv(output_filepath+'SMS_SC_SS_Segments.csv', index = False)

D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


                             name channel  status  count  \
0                          461243     sms  active  11531   
1          526 rto-JULY22-CLK-hzb     sms  active   1283   
2                   ARM CR 460939     sms  active  62935   
3               ARM.CR Last Month     sms  active  26541   
4                   ARMSMPL461149     sms  active   8279   
..                            ...     ...     ...    ...   
304                RTO-JULY22-CLK     sms  active   7579   
305               test-2565371505     sms  active   4255   
306  viewrenttoownhomes URL check     sms  active   6386   
307                      ZP461235     sms  active  85116   
308                 ZP Last Month     sms  active   4110   

                last_counted       Date  
0    2023-05-01 00:06:08 EST 2023-05-17  
1    2023-05-17 06:18:34 EST 2023-05-17  
2    2023-05-01 19:42:33 EST 2023-05-17  
3    2023-04-29 06:55:26 EST 2023-05-17  
4    2023-04-29 04:34:56 EST 2023-05-17  
..                       ..

                              name channel  status  count  \
0                           461243     sms  active  11531   
1           526 rto-JULY22-CLK-hzb     sms  active    946   
2                    ARMSMPL461149     sms  active   8279   
3                ARMSYF Last Month     sms  active      0   
4                   ARMUSACF461148     sms  active   2690   
...                            ...     ...     ...    ...   
2510                RTO-JULY22-CLK     sms  active   7579   
2511               test-2565371505     sms  active   4255   
2512  viewrenttoownhomes URL check     sms  active   6386   
2513                      ZP461235     sms  active  85116   
2514                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-05-01 00:06:08 EST 2023-05-26  
1     2023-05-26 04:31:13 EST 2023-05-26  
2     2023-04-29 04:34:56 EST 2023-05-26  
3     2023-04-29 05:14:10 EST 2023-05-26  
4     2023-04-29 03:27:33 EST 2023-05-26  
...      

                              name channel  status  count  \
0                           461243     sms  active  11531   
1           526 rto-JULY22-CLK-hzb     sms  active    940   
2                    ARMSMPL461149     sms  active   8279   
3                   ARMUSACF461148     sms  active   2690   
4                     B2CGC 461122     sms  active   7417   
...                            ...     ...     ...    ...   
6568                RTO-JULY22-CLK     sms  active   7579   
6569               test-2565371505     sms  active   4255   
6570  viewrenttoownhomes URL check     sms  active   6386   
6571                      ZP461235     sms  active  85116   
6572                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-05-01 00:06:08 EST 2023-06-15  
1     2023-06-15 04:47:06 EST 2023-06-15  
2     2023-04-29 04:34:56 EST 2023-06-15  
3     2023-04-29 03:27:33 EST 2023-06-15  
4     2023-05-01 13:44:37 EST 2023-06-15  
...      

                              name channel  status  count  \
0           526 rto-JULY22-CLK-hzb     sms  active    943   
1                    ARMSMPL461149     sms  active   8279   
2                   ARMUSACF461148     sms  active   2690   
3                     B2CGC 461122     sms  active   7417   
4                B2USRA Last Month     sms  active     36   
...                            ...     ...     ...    ...   
8308                RTO-JULY22-CLK     sms  active   7579   
8309               test-2565371505     sms  active   4255   
8310  viewrenttoownhomes URL check     sms  active   6386   
8311                      ZP461235     sms  active  85116   
8312                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-06-27 08:16:03 EST 2023-06-27  
1     2023-04-29 04:34:56 EST 2023-06-27  
2     2023-04-29 03:27:33 EST 2023-06-27  
3     2023-05-01 13:44:37 EST 2023-06-27  
4     2023-04-29 07:33:25 EST 2023-06-27  
...      

...

                               name channel  status  count  \
0            526 rto-JULY22-CLK-hzb     sms  active    889   
1          ALL HZB_RHD.CC_all codes     sms  active  25867   
2                     ARMSMPL461149     sms  active   8279   
3                    ARMUSACF461148     sms  active   2690   
4                      B2CGC 461122     sms  active   7417   
...                             ...     ...     ...    ...   
44109                RTO-JULY22-CLK     sms  active   7579   
44110               test-2565371505     sms  active   4255   
44111  viewrenttoownhomes URL check     sms  active   6386   
44112                      ZP461235     sms  active  85116   
44113                 ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2023-12-04 06:06:25 EST 2023-12-04  
1      2023-12-04 12:03:53 EST 2023-12-04  
2      2023-04-29 04:34:56 EST 2023-12-04  
3      2023-04-29 03:27:33 EST 2023-12-04  
4      2023-05-01 13:44:37 EST 2023

                               name channel  status  count  \
0            526 rto-JULY22-CLK-hzb     sms  active    872   
1          ALL HZB_RHD.CC_all codes     sms  active  27579   
2                     ARMSMPL461149     sms  active   8279   
3                    ARMUSACF461148     sms  active   2690   
4                      B2CGC 461122     sms  active   7417   
...                             ...     ...     ...    ...   
48523                RTO-JULY22-CLK     sms  active   7579   
48524               test-2565371505     sms  active   4255   
48525  viewrenttoownhomes URL check     sms  active   6386   
48526                      ZP461235     sms  active  85116   
48527                 ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-01-02 04:27:03 EST 2024-01-02  
1      2024-01-02 07:23:18 EST 2024-01-02  
2      2023-04-29 04:34:56 EST 2024-01-02  
3      2023-04-29 03:27:33 EST 2024-01-02  
4      2023-05-01 13:44:37 EST 2024

                                 name channel  status  count  \
0              526 rto-JULY22-CLK-hzb     sms  active    871   
1            ALL HZB_RHD.CC_all codes     sms  active  27937   
2      Copy HZB October 2022 Clickers     sms  active      0   
3                             DSS ALL     sms  active  31704   
4                             DSS-ALL     sms  active  31718   
...                               ...     ...     ...    ...   
52777                  RTO-JULY22-CLK     sms  active   7579   
52778                 test-2565371505     sms  active   4255   
52779    viewrenttoownhomes URL check     sms  active   6386   
52780                        ZP461235     sms  active  85116   
52781                   ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-01-18 06:09:05 EST 2024-01-18  
1      2024-01-18 02:59:17 EST 2024-01-18  
2      2024-01-18 01:53:40 EST 2024-01-18  
3      2024-01-18 05:09:20 EST 2024-01-18  
4      2024

                                 name channel  status  count  \
0              526 rto-JULY22-CLK-hzb     sms  active    871   
1            ALL HZB_RHD.CC_all codes     sms  active  28704   
2      Copy HZB October 2022 Clickers     sms  active      0   
3                             DSS ALL     sms  active  36221   
4                             DSS-ALL     sms  active  36224   
...                               ...     ...     ...    ...   
55383                  RTO-JULY22-CLK     sms  active   7579   
55384                 test-2565371505     sms  active   4255   
55385    viewrenttoownhomes URL check     sms  active   6386   
55386                        ZP461235     sms  active  85116   
55387                   ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-01-29 11:10:40 EST 2024-01-29  
1      2024-01-29 08:49:02 EST 2024-01-29  
2      2024-01-29 09:04:50 EST 2024-01-29  
3      2024-01-29 09:57:13 EST 2024-01-29  
4      2024

                                 name channel  status  count  \
0              526 rto-JULY22-CLK-hzb     sms  active    865   
1            ALL HZB_RHD.CC_all codes     sms  active  29364   
2      Copy HZB October 2022 Clickers     sms  active      0   
3                             DSS ALL     sms  active  40057   
4                             DSS-ALL     sms  active  40059   
...                               ...     ...     ...    ...   
59877                  RTO-JULY22-CLK     sms  active   7579   
59878                 test-2565371505     sms  active   4255   
59879    viewrenttoownhomes URL check     sms  active   6386   
59880                        ZP461235     sms  active  85116   
59881                   ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-02-14 09:39:57 EST 2024-02-14  
1      2024-02-14 15:55:16 EST 2024-02-14  
2      2024-02-14 05:31:26 EST 2024-02-14  
3      2024-02-14 07:47:41 EST 2024-02-14  
4      2024

## Update Post Data

In [42]:
# Output dailt post data
legacy_post = pd.read_csv(output_filepath+'SMSposts.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_post.to_csv(input_filepath+'SMSposts_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_post = pd.read_csv(input_filepath+'SMSPosts.csv',index_col=False)
drop = updated_post['Pub Name'].str.contains('SMS', case = False, na = False)
updated_post = updated_post.loc[drop]
today = date.today()
first_date_of_month = today.replace(day=1)
updated_post = updated_post.loc[updated_post['Pub Id'].isna() == False]
current_year = datetime.now().year
updated_post['Dates'] = pd.to_datetime(updated_post['Dates'], format="%m/%d").apply(lambda x: x.replace(year=current_year))
updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
updated_post = updated_post[updated_post['Dates']<=yesterday]
#updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates'] )
min_date = updated_post['Dates'].min()
legacy_post = legacy_post[legacy_post['Dates']<min_date]
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates']).dt.strftime('%m/%d/%Y')
post = [legacy_post, updated_post]
output_post= pd.concat(post, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_post)
output_post['Pub Id'] = output_post['Pub Id'].astype(str)
output_post['Pub Name'] = output_post['Pub Name'].astype(str)
output_post = output_post.loc[output_post['Pub Name'].str.contains('SMS', case = False, na = False)]
output_post['Dates'] =  pd.to_datetime(output_post['Dates']) 
output_post.to_csv(output_filepath+'SMSposts.csv', index = False)

            Dates    Pub Id                           Pub Name  Raw Posts  \
0      01/01/2022  461227.0       LM_SMS_Edemographic_247Loans          0   
1      01/01/2022  461313.0  LM_SMS_NorthPointDirect_RentToOwn          0   
2      01/01/2022  460921.0            LM_SMS_Spark_CreditCard      17102   
3      01/01/2022  461147.0                 461147_ARM.SYF_SMS      15884   
4      01/01/2022  460917.0                   460917_JM.OP_SMS       9999   
...           ...       ...                                ...        ...   
35130  02/21/2024  461241.0                   461241_I.HIE_SMS          0   
35131  02/21/2024  461507.0     1 - SMS:complete-coverage.com﻿          0   
35132  02/21/2024  461285.0                   461285_I.GCT_SMS          0   
35133  02/21/2024  461840.0              LM_SMS_DMS_SimplyJobs          0   
35134  02/21/2024  461867.0                   461867_I.TFE_SMS          0   

       GeoIP Good  % IP Good  GeoIP Bad  % IP Bad  Goodhyg  % Good  ...  \


## Update Optout 

In [43]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsreport_filepath) if isfile(join(smsreport_filepath, f)) & ('SMS_SC_SS_Report' in f)]
""" 
most_recent_smsreport_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in smsreport_files])
smsreport_year = most_recent_smsreport_date.year
smsreport_month = most_recent_smsreport_date.month
segement_day = most_recent_smsreport_date.day

most_recent_smsreport_file = "SMS_SC_SS_Report{0}{1:0>2}{2:0>2}.csv".format(smsreport_year,smsreport_month,segement_day)

smsreport_report = pd.read_csv( smsreport_filepath+ most_recent_smsreport_file)

smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
smsreport_report['Shortcode Name'] = ''
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
legacy_smsreport = pd.read_csv(output_filepath+'SMS_SC_SS_Report.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_smsreport.to_csv(input_filepath+'SMS_SC_SS_Report_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_smsreport['Date'] = pd.to_datetime(legacy_smsreport['Date'])
min_date = smsreport_report['Date'].min()
legacy_smsreport = legacy_smsreport[legacy_smsreport['Date']<min_date]
full_smsreport = [legacy_smsreport, smsreport_report]
output_smsreport = pd.concat(full_smsreport, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_smsreport)
output_smsreport.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)
"""
df = pd.DataFrame()
for i in smsreport_files:
    smsreport_report = pd.read_csv(smsreport_filepath+i)
    most_recent_smsreport_date = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
    smsreport_report['Shortcode Name'] = ''
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('Dollar Sensei'),'Shortcode Name' ] ='DSS'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('SVT'),'Shortcode Name' ] ='SVT'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('UAA'),'Shortcode Name' ] ='UAA'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('PRC'),'Shortcode Name' ] ='PRC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('UAATF'),'Shortcode Name' ] ='UAATF'
    df = pd.concat([df,smsreport_report])
df.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)


# update apikey

In [44]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsapikey_filepath) if isfile(join(smsapikey_filepath, f)) & ('apikeyreport' in f)]

df = pd.DataFrame()
for i in smsreport_files:
    smsapikey_report = pd.read_csv(smsapikey_filepath+i)
    smsapikey_report['Date'] = pd.to_datetime(smsapikey_report['Date'])
    smsapikey_report['DP.SV'] = smsapikey_report['Apikey'].str.split(" |-",expand = True)[0]
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.FD',na= False),'DP.SV' ] ='I.FD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.RTOD',na= False),'DP.SV' ] ='I.RTOD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.LD',na= False),'DP.SV' ] ='I.LD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.NFD',na= False),'DP.SV' ] ='I.NFD'
    df = pd.concat([df,smsapikey_report])
df = df.drop_duplicates()
df.to_csv(output_filepath+'SMS_SC_SS_Apikey.csv', index = False)


## Update Flow

In [45]:
legacy_flows = pd.read_csv(output_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_flows.to_csv(input_filepath+'SMS_SC_SS_Flows_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_flows = pd.read_csv(input_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
legacy_flows['Period'] = pd.to_datetime(legacy_flows['Period'],format='mixed')
min_date = updated_flows['Period'].min()
legacy_flows = legacy_flows[legacy_flows['Period']<min_date]
flows = [legacy_flows, updated_flows]
output_flows = pd.concat(flows, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_flows)
output_flows.to_csv(output_filepath+'SMS_SC_SS_Flows.csv', index = False)


         Id                    Name               Period  \
0      5009           PA.SWP-MBC-PR  2023-03-06 00:00:00   
1      5009           PA.SWP-MBC-PR  2023-03-07 00:00:00   
2      5009           PA.SWP-MBC-PR  2023-03-07 00:00:00   
3      5009           PA.SWP-MBC-PR  2023-03-08 00:00:00   
4      5009           PA.SWP-MBC-PR  2023-03-08 00:00:00   
...     ...                     ...                  ...   
33638  5137  JET.ZTA-FLC-VZN-BU-SFW           2024-02-21   
33639  5137  JET.ZTA-FLC-VZN-BU-SFW           2024-02-21   
33640  5139        DUP-I.A4F-PR-SFW           2024-02-21   
33641  5139        DUP-I.A4F-PR-SFW           2024-02-21   
33642  5139        DUP-I.A4F-PR-SFW           2024-02-21   

                                          Shortcode                ListName  \
0      MyBenefitsClub :: MBC :: 80837 [80837 : 306]  MyBenefitsClub [12421]   
1      MyBenefitsClub :: MBC :: 80837 [80837 : 306]  MyBenefitsClub [12421]   
2      MyBenefitsClub :: MBC :: 80837 [808

## Update Creatives Data

In [46]:
legacy_creatives = pd.read_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_creatives.to_csv(input_filepath+'SMS_SC_SS_CreativesStats_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_creatives = pd.read_csv(input_filepath+'creativestats.csv',index_col=False)
legacy_creatives['Tstamp'] = pd.to_datetime(legacy_creatives['Tstamp'],format='mixed')
min_date = updated_creatives['Tstamp'].min()
legacy_creatives = legacy_creatives[legacy_creatives['Tstamp']<min_date]
creativestats = [legacy_creatives, updated_creatives]
output_creatives = pd.concat(creativestats, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_creatives)
output_creatives['Tstamp'] = pd.to_datetime(output_creatives['Tstamp'],format='mixed')
output_creatives.to_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv', index = False)


        JobId                     Tstamp                              Offer  \
0           0  2021-10-01 00:00:00+00:00                                NaN   
1      237891  2021-10-01 17:00:00+00:00  B2USRA - Cheris Studio Benefits A   
2           0  2021-10-02 00:00:00+00:00                                NaN   
3      238383  2021-10-02 17:00:00+00:00  B2USRA - Cheris Studio Benefits A   
4           0  2021-10-03 00:00:00+00:00                                NaN   
...       ...                        ...                                ...   
41000  641035       2024-02-21T18:00:57Z                  12710 - WI SS HZB   
41001  640859       2024-02-21T18:00:57Z                  8838 - W4 EDU UAA   
41002  640859       2024-02-21T18:00:57Z                  8838 - W4 EDU UAA   
41003  640859       2024-02-21T18:00:57Z                  8838 - W4 EDU UAA   
41004  641045       2024-02-21T20:00:08Z                  11600 - WI UG MBC   

                 Segment  CreativeId            Cre

In [47]:
output_creatives['Tstamp'].astype(str).str[:10].unique()

array(['2021-10-01', '2021-10-02', '2021-10-03', '2021-10-04',
       '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08',
       '2021-10-09', '2021-10-10', '2021-10-11', '2021-10-12',
       '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-16',
       '2021-10-17', '2021-10-18', '2021-10-19', '2021-10-20',
       '2021-10-21', '2021-10-22', '2021-10-23', '2021-10-24',
       '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28',
       '2021-10-29', '2021-10-30', '2021-10-31', '2021-11-01',
       '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05',
       '2021-11-06', '2021-11-07', '2021-11-08', '2021-11-09',
       '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-13',
       '2021-11-14', '2021-11-15', '2021-11-16', '2021-11-17',
       '2021-11-18', '2021-11-19', '2021-11-20', '2021-11-21',
       '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-25',
       '2021-11-26', '2021-11-27', '2021-11-28', '2021-11-29',
       '2021-11-30', '2021-12-01', '2021-12-02', '2021-

## Update Jobs

In [48]:
legacy_jobs = pd.read_csv(output_filepath+'SMS_SC_SS_Jobs.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_jobs.to_csv(input_filepath+'SMS_SC_SS_Jobs_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
updated_jobs = pd.read_csv(input_filepath+'jobs.csv',index_col=False)
legacy_jobs['scheduled_tstamp'] = pd.to_datetime(legacy_jobs['scheduled_tstamp'],format = "mixed")
min_date = updated_jobs['scheduled_tstamp'].min()
legacy_jobs = legacy_jobs[legacy_jobs['scheduled_tstamp']<min_date]
jobs = [legacy_jobs, updated_jobs]
output_jobs = pd.concat(jobs, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_jobs)
output_jobs.to_csv(output_filepath+'SMS_SC_SS_Jobs.csv', index = False)

           id                                      name  \
0      237891                     B2USRA_1 [2021-10-01]   
1      238383                     B2USRA_1 [2021-10-02]   
2      238832                     B2USRA_1 [2021-10-03]   
3      239249                     B2USRA_1 [2021-10-04]   
4      239695                     B2USRA_1 [2021-10-05]   
...       ...                                       ...   
32394  641094      SS_DSS_JET-ZTA-30DC_10086_CT_21Feb24   
32395  641096   SS_FLC_JET-ZTA-30DC-VZN_11468_P_21Feb24   
32396  641100         SS_SVT_AL-PL-30DC_10086_P_21Feb24   
32397  641103   SS_HZB_AL-PL-2-30DC-VZN_12760_P_21Feb24   
32398  641134  SS_HZB_ZM-PL-2-30DC-VZN_10086_CT_21Feb24   

                                   offer  shortcode             start_tstamp  \
0      B2USRA - Cheris Studio Benefits A      70610  2021-10-01 13:00:09 EST   
1      B2USRA - Cheris Studio Benefits A      70610  2021-10-02 13:00:10 EST   
2      B2USRA - Cheris Studio Benefits A      70610

In [49]:
output_jobs['start_tstamp'].str[:10].unique()

array(['2021-10-01', '2021-10-02', '2021-10-03', '2021-10-04',
       '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08',
       '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-16',
       '2021-10-17', '2021-10-18', '2021-10-19', '2021-10-20',
       '2021-10-21', '2021-10-22', '2021-10-23', '2021-10-24',
       '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28',
       '2021-10-29', '2021-10-30', '2021-10-31', '2021-11-01',
       '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05',
       '2021-11-06', '2021-11-07', '2021-11-08', '2021-11-09',
       '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15',
       '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19',
       '2021-11-20', '2021-11-21', '2021-11-22', '2021-11-23',
       '2021-11-24', '2021-11-25', '2021-11-26', '2021-11-27',
       '2021-11-28', '2021-11-29', '2021-11-30', '2021-12-01',
       '2021-12-02', '2021-12-03', '2021-12-04', '2021-12-05',
       '2021-12-06', '2021-12-07', '2021-12-08', '2021-

## Update Post Data

In [50]:
# Output dailt post data
legacy_post = pd.read_csv(output_filepath+'SMSposts.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_post.to_csv(input_filepath+'SMSposts_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_post = pd.read_csv(input_filepath+'SMSPosts.csv',index_col=False)
drop = updated_post['Pub Name'].str.contains('SMS', case = False, na = False)
updated_post = updated_post.loc[drop]
today = date.today()
first_date_of_month = today.replace(day=1)
updated_post = updated_post.loc[updated_post['Pub Id'].isna() == False]
current_year = datetime.now().year
updated_post['Dates'] = pd.to_datetime(updated_post['Dates'], format="%m/%d").apply(lambda x: x.replace(year=current_year))
updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
updated_post = updated_post[updated_post['Dates']<=yesterday]
#updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates'] )
min_date = updated_post['Dates'].min()
legacy_post = legacy_post[legacy_post['Dates']<min_date]
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates']).dt.strftime('%m/%d/%Y')
post = [legacy_post, updated_post]
output_post= pd.concat(post, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_post)
output_post['Pub Id'] = output_post['Pub Id'].astype(str)
output_post['Pub Name'] = output_post['Pub Name'].astype(str)
output_post = output_post.loc[output_post['Pub Name'].str.contains('SMS', case = False, na = False)]
output_post['Dates'] =  pd.to_datetime(output_post['Dates']) 
output_post.to_csv(output_filepath+'SMSposts.csv', index = False)

            Dates    Pub Id                           Pub Name  Raw Posts  \
0      01/01/2022  461227.0       LM_SMS_Edemographic_247Loans          0   
1      01/01/2022  461313.0  LM_SMS_NorthPointDirect_RentToOwn          0   
2      01/01/2022  460921.0            LM_SMS_Spark_CreditCard      17102   
3      01/01/2022  461147.0                 461147_ARM.SYF_SMS      15884   
4      01/01/2022  460917.0                   460917_JM.OP_SMS       9999   
...           ...       ...                                ...        ...   
35130  02/21/2024  461241.0                   461241_I.HIE_SMS          0   
35131  02/21/2024  461507.0     1 - SMS:complete-coverage.com﻿          0   
35132  02/21/2024  461285.0                   461285_I.GCT_SMS          0   
35133  02/21/2024  461840.0              LM_SMS_DMS_SimplyJobs          0   
35134  02/21/2024  461867.0                   461867_I.TFE_SMS          0   

       GeoIP Good  % IP Good  GeoIP Bad  % IP Bad  Goodhyg  % Good  ...  \


In [51]:
output_post['Dates'].unique()

<DatetimeArray>
['2022-01-01 00:00:00', '2022-01-02 00:00:00', '2022-01-03 00:00:00',
 '2022-01-04 00:00:00', '2022-01-05 00:00:00', '2022-01-06 00:00:00',
 '2022-01-07 00:00:00', '2022-01-08 00:00:00', '2022-01-09 00:00:00',
 '2022-01-10 00:00:00',
 ...
 '2024-02-12 00:00:00', '2024-02-13 00:00:00', '2024-02-14 00:00:00',
 '2024-02-15 00:00:00', '2024-02-16 00:00:00', '2024-02-17 00:00:00',
 '2024-02-18 00:00:00', '2024-02-19 00:00:00', '2024-02-20 00:00:00',
 '2024-02-21 00:00:00']
Length: 775, dtype: datetime64[ns]

## Update Offers

In [52]:
offers_old = pd.read_csv(output_filepath + 'SMS_SC_SS_Offers.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong
offers_old.to_csv(input_filepath+'SMS_SC_SS_Offers_backup.csv', index = False)
# This is the offer raw data downloaded by the outsource 
offers = pd.read_csv(input_filepath+'SMS_SC_SS_Offers.csv',index_col=False)
offers['hitpath_offer_id'] = offers['name'].str.split(' ',expand = True)[0].str.strip()
offers = offers[['id', 'name','hitpath_offer_id', 'type', 'status', 'redirect_type', 'conversion_event',
       'conversion_payout', 'currency']]
print(offers.head(5))
offers.to_csv(output_filepath+'SMS_SC_SS_Offers.csv', index = False)

      id                name hitpath_offer_id type  status redirect_type  \
0  20267   6963 - WM UBG CSS             6963  sms  active          HTTP   
1  40816   12088 - BE OE HZB            12088  sms  active          HTTP   
2  48116  12610 - AVL CL SVT            12610  sms  active          HTTP   
3  48117  12611 - AVL QL HZB            12611  sms  active          HTTP   
4  40935   12163 - BK OD HZB            12163  sms  active          HTTP   

  conversion_event  conversion_payout currency  
0              CPA                  0      USD  
1              CPA                  0      USD  
2              CPA                  0      USD  
3              CPA                  0      USD  
4              CPA                  0      USD  
